In [ ]:
import os
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes
from minimal.solver import Solver
from minimal import armatures
from minimal.models import KinematicModel, KinematicPCAWrapper
import numpy as np
import minimal.config as config
from dataloader.result_loader import KinectResultLoader
from minimal.bridge import JointsBridge

from tqdm.notebook import tqdm
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

from dataloader.utils import ymdhms_time

%load_ext autoreload
%autoreload 2

np.random.seed(20160923)

In [ ]:
# pose_glb = np.zeros([1, 3]) # global rotation


########################## smpl settings ##########################
# note that in smpl and smpl-h no pca for pose is provided
# therefore in the model we fake an identity matrix as the pca coefficients
# to make the code compatible

n_pose = 23 * 3 # degrees of freedom, (n_joints - 1) * 3
# smpl 1.0.0: 10
# smpl 1.1.0: 300
n_shape = 10
# TODO: Read pose from skeleton
k_loader = KinectResultLoader('./ignoredata/minimal_files/input/')
k_loader.file_dict.pop("kinect/master/pcls")
files = k_loader.select_item_in_tag(58, "id", "kinect/master/skeleton")
kinect_skeleton = np.load(files["filepath"])

bridge = JointsBridge()
keypoints_gt, _ = bridge.smpl_from_kinect(kinect_skeleton[0])

In [ ]:
mesh = KinematicModel(config.SMPL_MODEL_1_0_PATH, armatures.SMPLArmature, scale=1)

########################## solving example ############################

wrapper = KinematicPCAWrapper(mesh, n_pose=n_pose)
solver = Solver(wrapper, verbose=True, max_iter=int(10e7))

# mesh_gt, keypoints_gt = \
# mesh.set_params(pose_pca=pose_pca, pose_glb=pose_glb, shape=shape)
# pointcloud_gt = Pointclouds(sample_points_from_meshes(mesh_gt, num_samples=1000))
pointcloud_gt = None
params_est = solver.solve_full(keypoints_gt, pointcloud_gt)

shape_est, pose_pca_est, pose_glb_est = wrapper.decode(params_est)

print('----------------------------------------------------------------------')
print('estimated parameters')
print('pose pca coefficients:', pose_pca_est)
print('pose global rotation:', pose_glb_est)
print('shape: pca coefficients:', shape_est)

mesh.set_params(pose_pca=pose_pca_est)
mesh.save_obj(os.path.join(config.SAVE_PATH, './esttm={}.obj'.format(ymdhms_time())))